Install dependencies

In [8]:
!pip install neo4j
!pip install requests

First create the FHIR data types. Need this before anything else.


In [20]:
from neo4j import GraphDatabase
import requests
import json

def clear(tx):
    tx.run("MATCH (n) DETACH DELETE n")

driver = GraphDatabase.driver("neo4j://3.83.17.192:7687", auth=("neo4j", "troubleshooters-decreases-helicopters"))

response = requests.get('https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/fhir_data_type_nodes.json')
the_nodes = json.loads(response.text)
response = requests.get('https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/fhir_data_type_relationships.json')
the_relationships = json.loads(response.text)

with driver.session() as session:
    session.write_transaction(clear)
    for key, value in the_nodes.items():
        query = "UNWIND $nodes as data CREATE (n:%s) SET n = data;" % (key)
        print(query)
        result = session.run(query, nodes=value)

    for key, value in the_relationships.items():
        query = "UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:%s]->(m)" % (key)
        print(query)
        result = session.run(query, rels=value)

driver.close()

UNWIND $nodes as data CREATE (n:FHIR) SET n = data;
UNWIND $nodes as data CREATE (n:WEB_SOURCE) SET n = data;
UNWIND $nodes as data CREATE (n:FHIR_DATA_TYPE) SET n = data;
UNWIND $nodes as data CREATE (n:FHIR_DATA_TYPE_PROPERTY) SET n = data;
UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:FROM_SOURCE]->(m)
UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:HAS_DATA_TYPE]->(m)
UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:HAS_DATA_TYPE_PROPERTY]->(m)


Now check we have loaded ok. Check the FHIR version, should be 4.0.1

In [21]:
with driver.session() as session:
  query = """MATCH (n:FHIR) Return n.version as version""" 
  result = session.run(query)
  for record in result:
    print("Version: ", record["version"])

driver.close()

Version:  4.0.1


In [22]:
response = requests.get('https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/canonical_model_nodes.json')
the_nodes = json.loads(response.text)
response = requests.get('https://raw.githubusercontent.com/data4knowledge/biomedical_concepts/main/data/canonical_model_relationships.json')
the_relationships = json.loads(response.text)

with driver.session() as session:
    for key, value in the_nodes.items():
        query = "UNWIND $nodes as data CREATE (n:%s) SET n = data;" % (key)
        print(query)
        result = session.run(query, nodes=value)

    for key, value in the_relationships.items():
        query = "UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:%s]->(m)" % (key)
        print(query)
        result = session.run(query, rels=value)

driver.close()

UNWIND $nodes as data CREATE (n:CANONICAL_NODE) SET n = data;
UNWIND $rels as data MATCH (n {uri: data.from}) MATCH (m {uri: data.to}) CREATE (n)-[:CONSISTS_OF]->(m)
